In [ ]:
pip install -qU langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY']= os.getenv('TAVILY_API_KEY')


# 상견례 장소 추천

프롬프트

In [3]:
PROMPT_TEMPLATE_eg = """
# INSTRUCTION
You are an AI assistant providing recommendations for engagement meeting places based on a specific location requested by the user. Your goal is to respond accurately and courteously to the user's queries, suggesting up to five venues and providing a brief description of each.

# Tools 
- Assistant has access to the following tools:

{tools}

- To use a tool, please use the following format:

Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, MUST be related to venue location and details
Observation: the result of the action
...the Thought/Action/Action Input/Observation sequence cannot repeat several times

- When you have a response to say to the user, or if you do not need to use a tool, you MUST use the format:

Thought: No
Final Answer: [List up to five recommended venues and provide a brief description for each]

# RESULTS 
When suggesting venues, provide information based on the following criteria:

    Location: The geographical location of the venue.
    Capacity: How many people the venue can accommodate.
    Atmosphere: The general ambiance and style of the venue.
    Special Features: Any unique features or services that the venue offers.
    Accessibility: Ease of access for guests, including parking and public transport options.

Deliver the venue information in a way that is easy for the user to understand. If necessary, briefly provide additional tips related to organizing engagement meetings.

# IMPORTANT
- Always respond in Korean.
- If the user does not specify a location, do not provide venue recommendations but ask for more details about the location.

# START!

Previous conversation history:
{chat_history}

New input: {input}

{agent_scratchpad}

"""


In [4]:
import os
import readline
import json
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain_community.tools.tavily_search.tool import TavilySearchResults
from langchain.tools.render import render_text_description
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.prompts import PromptTemplate


def custom_input(prompt):
    readline.set_pre_input_hook(lambda: readline.insert_text(""))
    readline.set_startup_hook(lambda: readline.insert_text(""))
    try:
        return input(prompt)
    finally:
        readline.set_pre_input_hook(None)
        readline.set_startup_hook(None)


def main():

    memory = ConversationBufferMemory(memory_key="chat_history", input_key="input") #  #

    tools = [TavilySearchResults(max_results=4)]


    prompt = PromptTemplate(
        input_variables=["agent_scratchpad", "chat_history", "input", "tools", "tool_names"],
        template=PROMPT_TEMPLATE_eg #프롬프트
        )
    
    prompt = prompt.partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
    llm_with_stop = llm.bind(stop=["\nObservation"])

    agent = (
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
            "chat_history": lambda x: x["chat_history"]
        }
        | prompt
        | llm_with_stop
        | ReActSingleInputOutputParser()
    )
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )

    while True:
        user_input = custom_input("당신: ")
        if user_input.lower() == "exit":
            print("어시스턴트: 안녕히 가세요!")
            break

        try:
            response = agent_executor.invoke({
                "input": user_input
            })
            print("어시스턴트: " + response["output"])

        except Exception as e:
            print(f"어시스턴트: 죄송합니다. 오류가 발생했습니다: {str(e)}")


if __name__ == "__main__":
    main()


/var/folders/k3/f4f9x_vx7zg0vcl_gmq4gd0c0000gn/T/ipykernel_2191/2658372823.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", input_key="input") #  #




> Entering new AgentExecutor chain...
Thought: Yes  
Action: tavily_search_results_json  
Action Input: "동작구 상견례 장소"  [{'url': 'https://rengar-yuumi.com/13', 'content': '상견례 장소 고르기 상견례는 공식적인 자리인 만큼 어디서 진행할지 고민하시는 분들이 많을 거 같습니다. ※상견례 장소 추천 1순위 고려사항 : 코스요리, 룸식당, 음식 맛, 5만원~10만원 사이의 가격 위의 내용을 충족하는 식당만 추천드리겠습니다. 진진수라'}, {'url': 'https://m.blog.naver.com/peacefuldays0/222891282521', 'content': '상견례 테이블 세팅이 세련되고 예쁨 (원앙, 푸른 비단) → 아선재는 상견례 장소로 검색했을 때 나오는 식당은 아니지만, 가족들과 자주 방문하던 식당이고 워낙 음식 퀄리티나 인테리어가 괜찮아서 넣어봤다.'}, {'url': 'https://weddinglast.com/서울-상견례-식당-장소-추천-top-10/', 'content': '서울 상견례 식당을 찾고 있다면 서울 내에서 접근성이 뛰어나고 많은 사람들이 방문하는 상견례 식당 추천 TOP 10을 참고하시길 바랍니다. 상견례 장소는 혼담을 나누고 양가 어르신들이 처음 만나는 장소이기 때문에 정갈하고 조용한 식당을 선호하는 경우가 많습니다. 서울 내에서 상견례를 진행'}, {'url': 'https://post.naver.com/viewer/postView.nhn?volumeNo=24263433', 'content': "36년 전통을 자랑하는 상견례 명소 다봉은 '봉우리가 많다'는 뜻으로 하늘로 솟은 봉우리의 힘찬 기개와 다복을 의미하는 이름 덕분에 이곳에서 상견례를 하면 다복한 가정을 이룬다는 속설과 함께 높은 인기를 자랑하는 고급 일식당이다. 7개의 다다미방과 5개의 테이블룸이 준비되어 있어 차분하고"}]Thoug